In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4

cpu


In [2]:
with open("wizard_of_oz.txt", 'r', encoding='utf-8') as f:
    text = f.read()
print(text[:200])

  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ, OZMA OF OZ, ETC.

  ILLUSTRATED BY JOHN R. NEILL

  BOOKS OF WONDER WILLIAM MORROW & CO., INC. NEW


In [3]:
chars = sorted(set(text))
vocab_size = len(chars)
print(chars)
print(vocab_size)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [4]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda w: [string_to_int[c] for c in w]
decode = lambda n: ''.join([int_to_string[i] for i in n])

In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [6]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [7]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
print(x)
print(y)

tensor([80,  1,  1, 28, 39, 42, 39, 44])
tensor([ 1,  1, 28, 39, 42, 39, 44, 32])


In [8]:
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context}, target is {target}.")

When input is tensor([80]), target is 1.
When input is tensor([80,  1]), target is 1.
When input is tensor([80,  1,  1]), target is 28.
When input is tensor([80,  1,  1, 28]), target is 39.
When input is tensor([80,  1,  1, 28, 39]), target is 42.
When input is tensor([80,  1,  1, 28, 39, 42]), target is 39.
When input is tensor([80,  1,  1, 28, 39, 42, 39]), target is 44.
When input is tensor([80,  1,  1, 28, 39, 42, 39, 44]), target is 32.


In [9]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs', x.shape, x)
print('targets', y.shape, y)

tensor([29350, 13776, 80683, 17736])
inputs torch.Size([4, 8]) tensor([[54,  1, 65, 62, 73, 73, 65, 58],
        [57,  9,  1, 54, 67, 57,  1, 61],
        [58, 75, 58, 71,  9,  1, 59, 68],
        [ 1, 73, 61, 54, 73,  9,  1, 76]])
targets torch.Size([4, 8]) tensor([[ 1, 65, 62, 73, 73, 65, 58,  1],
        [ 9,  1, 54, 67, 57,  1, 61, 58],
        [75, 58, 71,  9,  1, 59, 68, 71],
        [73, 61, 54, 73,  9,  1, 76, 58]])


In [21]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :]   # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)   # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1)    # (B, 1)
            # append sampled index to running sequence
            index = torch.cat((index, index_next), dim=1)   # (B, T+1)
        return index


In [22]:
model = BigramLanguageModel(vocab_size=vocab_size)
m = model.to(device=device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


MEAtGw8wdF
16kUF(kZ:xNNf.8Wu.]Sn8sax&45cj!﻿Z?lhty)pk8Q,&JkL2Mcfx﻿n﻿3m0eVv1YN1Yd_lh_q67GyS5T4hLq6G!*K gGqn,:!2y*pXeuTS1fDtC,:X]iom,xlQyCW!gg,Qy)AxRAzx;H6, ][97y-J?_n
.d8ibc"&c]UMvVPJPMvPhAWkXC0eikwy"P(kwe7x!T4-l&5o:*(EiPf]e)vnu_j5*Fydb;g-u"TLp)In]Z&qKbN4Ei])G4(f-Oj[J'mu:﻿
]o(W3iJq]UCEZ&JQaBRD1Yjc7W1WLE(FkL'﻿pBA)6sEA-l(b-﻿(F;gTANxfY50F(;P
TVw5 ]h)v0ZCtu
m*F6.HK"V9;Ho*﻿h*]o k﻿-Gg,FHs0W5mDw'wgFz1(VBv0;bZLYB8yvJwZrc7YuTusZ:N JL*7DySzStaMvqZ97Cd_6GMx_."YR!UCc_p)mkUWmt3HaD3)KX"FumA-L)mhC)Ef; KeX4Z-xQO6
